In [1]:
import numpy as np
import pandas as pd
import sys
import os 

from os import path
from eppy import eppy
from eppy import hvacbuilder
from eppy import modeleditor
from eppy.modeleditor import IDF


In [2]:
# this function determines what idf version and connects the idd for eppy
def eppy_connection(idffile):
    fp = open(idffile)
    lines=fp.readlines()
    vers = ""
    vers = vers.join(lines[2:12])
    vers
    if vers.find('8.7') != -1:
        print('Using EnergyPlus version 8.7')
        iddfile = 'C:\\EnergyPlusV8-7-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('8.8') != -1:
        print('Using EnergyPlus version 8.8')
        iddfile = 'C:\\EnergyPlusV8-8-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('8.9') != -1:
        print('Using EnergyPlus version 8.9')
        iddfile = 'C:\\EnergyPlusV8-9-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('9.0') != -1:
        print('Using EnergyPlus version 9.0.1')
        iddfile = 'C:\\EnergyPlusV9-0-1\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    else:
        print('IDF Type not found on line 3')

In [3]:
#define paths
commonSlab_path = os.path.join(path.abspath(path.join('',"../..")), "DOE_LRMF\\SEED_Models\\common_hp_slab_IECC_2012_V8.7.idf")
commonBsmt_path = os.path.join(path.abspath(path.join('',"../..")), "DOE_LRMF\\SEED_Models\\common_hp_heatedbsmtIECC_2012_V8.7.idf")
gardenSlab_path = os.path.join(path.abspath(path.join('',"../..")), "DOE_LRMF\\SEED_Models\\garden_hp_slab_IECC_2012_V8.7.idf")
gardenBsmt_path = os.path.join(path.abspath(path.join('',"../..")), "DOE_LRMF\\SEED_Models\\garden_hp_heatedbsmtIECC_2012_V8.7.idf")

#make connections
commonSlab = eppy_connection(commonSlab_path)
commonBsmt = eppy_connection(commonBsmt_path)
gardenSlab = eppy_connection(gardenSlab_path)
gardenBsmt = eppy_connection(gardenBsmt_path)

# create groups
idfs = [commonSlab, commonBsmt, gardenSlab, gardenBsmt]
common = [commonSlab, commonBsmt]
garden = [gardenSlab, gardenBsmt]
bsmt = [commonBsmt, gardenBsmt]
slab = [commonSlab, gardenSlab]

Using EnergyPlus version 8.7
Using EnergyPlus version 8.7
Using EnergyPlus version 8.7
Using EnergyPlus version 8.7


# Disable Ground Heat Transfer Control Preprocessor

In [4]:
controlnames = ['gtp_control']
for i in range(0,len(idfs)):
    # grab groundtemp from list of control name
    ctlname = idfs[i].idfobjects['GroundHeatTransfer:Control']
    groundtemp = [x for x in ctlname if x.Name == controlnames[0]][0]
    groundtemp.Run_Basement_Preprocessor = 'no'
    groundtemp.Run_Slab_Preprocessor = 'no'
   
    print(groundtemp.Run_Basement_Preprocessor)
    print(groundtemp.Run_Slab_Preprocessor)


no
no
no
no
no
no
no
no


# Change slab boundary condition

In [5]:
## LIST BOUNDARY CONDITION TO MODIFY

outsideconditions = ['GroundBasementPreprocessorAverageFloor', 'GroundBasementPreprocessorAverageWall', 'GroundSlabPreprocessorAverage']


In [6]:
for i in range(0,len(idfs)):
    surfaces = idfs[i].idfobjects['BuildingSurface:Detailed']
    for j in range(0,len(outsideconditions)):
        boundary = [x for x in surfaces if x.Outside_Boundary_Condition == outsideconditions[j]]
#         print(boundary)
        if len(boundary) == 0:
            continue  
        for k in range(0,len(boundary)):
#             print(boundary[k].Outside_Boundary_Condition)
            boundary[k].Outside_Boundary_Condition = 'Ground'
            print('Boundary condition: '+boundary[k].Name+' change to '+boundary[k].Outside_Boundary_Condition) 



Boundary condition: Floor_unit1_FrontRow_BottomFloor change to Ground
Boundary condition: Floor_unit2_FrontRow_BottomFloor change to Ground
Boundary condition: Floor_unit3_FrontRow_BottomFloor change to Ground
Boundary condition: Floor_unit1_BackRow_BottomFloor change to Ground
Boundary condition: Floor_unit2_BackRow_BottomFloor change to Ground
Boundary condition: Floor_unit3_BackRow_BottomFloor change to Ground
Boundary condition: g Floor C change to Ground
Boundary condition: BGFloor change to Ground
Boundary condition: BGWall_lower_ldf change to Ground
Boundary condition: BGWall_lower_sdr change to Ground
Boundary condition: BGWall_lower_ldb change to Ground
Boundary condition: BGWall_lower_sdl change to Ground
Boundary condition: Floor_unit1_FrontRow_BottomFloor change to Ground
Boundary condition: Floor_unit2_FrontRow_BottomFloor change to Ground
Boundary condition: Floor_unit3_FrontRow_BottomFloor change to Ground
Boundary condition: Floor_unit1_BackRow_BottomFloor change to Gro

# Remove Output:Variable and Meters


In [7]:
# Remove Output:Variable

for i in range(len(idfs)):
    OutputVariables = idfs[i].idfobjects['Output:Variable']
    for j in range(len(OutputVariables)):
        idfs[i].removeidfobject(OutputVariables[0])


In [8]:
# Remove all Output:Meter

for i in range(len(idfs)):
    OutputVariables = idfs[i].idfobjects['Output:Meter']
    for j in range(len(OutputVariables)):
        idfs[i].removeidfobject(OutputVariables[0])


# Change default ventilaiton flow rate of Common corridors 
### (Designed flow: 40 cfm)

In [9]:
## list corridors
corridorzones = ['Ventilation_G Corridor', 'Ventilation_M Corridor', 'Ventilation_T Corridor']

for i in range(len(common)):
    ZoneVents = common[i].idfobjects['ZoneVentilation:DesignFlowRate']
    for j in range(len(corridorzones)):
        # grab corridors from list of zones
        corridor = [x for x in ZoneVents if x.Name == corridorzones[j]]
        corridor[0].Design_Flow_Rate = '0.018877898'  # 40 cfm
        #display changes
        print( corridor[0].Name +'\'s design flow rate was changed to '+ corridor[0].Design_Flow_Rate) 
    

Ventilation_G Corridor's design flow rate was changed to 0.018877898
Ventilation_M Corridor's design flow rate was changed to 0.018877898
Ventilation_T Corridor's design flow rate was changed to 0.018877898
Ventilation_G Corridor's design flow rate was changed to 0.018877898
Ventilation_M Corridor's design flow rate was changed to 0.018877898
Ventilation_T Corridor's design flow rate was changed to 0.018877898


# Change default ventilaiton flow rate of basements
### (Designed flow for storage: 500 cfm)

In [10]:
## list basement
bsmtzones = ['Ventilation_unitheatedbsmt']

for i in range(len(bsmt)):
    ZoneVents = bsmt[i].idfobjects['ZoneVentilation:DesignFlowRate']
    for j in range(len(bsmtzones)):
        # grab corridors from list of zones
        basementvent = [x for x in ZoneVents if x.Name == bsmtzones[j]]
        basementvent[0].Design_Flow_Rate = '0.235973725'  # 500 cfm
        #display changes
        print( basementvent[0].Name +'\'s design flow rate was changed to '+ basementvent[0].Design_Flow_Rate) 

Ventilation_unitheatedbsmt's design flow rate was changed to 0.235973725
Ventilation_unitheatedbsmt's design flow rate was changed to 0.235973725


# Change Output:Meter Reporting Frequency to Annual

In [11]:
# for i in range(len(idfs)):
#     meters = idfs[i].idfobjects['Output:Meter']
#     for j in range(len(meters)):
#         meters[j].Reporting_Frequency = 'Annual'

# Add Electricity and Gad annual meters


In [12]:
for i in range(len(idfs)):
    newobject = idfs[i].newidfobject("Output:Meter")
    newobject.Name = 'Electricity:Facility'
    newobject.Reporting_Frequency = 'Annual'
    
    newobject = idfs[i].newidfobject("Output:Meter")
    newobject.Name = 'Gas:Facility'
    newobject.Reporting_Frequency = 'Annual'



In [13]:
## save idfs
[x.save() for x in idfs]

[None, None, None, None]